# Assignment 1

In this assignment you will use this starter script to play around with 500 mb height data using xarray, a powerful package in Python that makes NetCDF files incredibly accessible. Each cell contains both code and comments. Comments begin with a "#" or ''' symbol and tells python to ignore that line. The reason you do not see any "#" symbols in this cell is because it is a "markdown" cell, which is intended only for text. Note: if you see "x=" followed by ''', it is solely to prevent the comment from being displayed as output. You can ignore this.  

**Be sure to read all directions, as some code will be commented out initially. You will need to uncomment these lines to run the script properly. All sections that need to be uncommented will explicitly say *uncomment*.**

Submit a final writeup (roughly 1-2 detailed paragraphs) on Canvas along with your script. Your writeup will need to discuss the following:

1. What type of data are these? Observations? Something else? Explain what this means.
2. Describe the differences between your monthly, climatology, and anomaly plots. 
3. Discuss why anomaly plots might be more useful than the monthly plot. 
4. Describe any coding issues you encountered. 

Be sure to include your images in your report. 

### Obtaining the data:

1. From the **Weather and Climate Resources** link on Canvas, go to the **IRI/LDEO Climate Data Library**.
2. Select **Data by Source > NOAA > NCEP-NCAR > CDAS-1 > MONTHLY > Intrinsic > Pressure Level > Geopotential Height**.
3. Go to the **Data Selection** tab. Restrict your data to include the 500 mb surface between Jan 1991 and December 2020. Click **Restrict Ranges** and **Stop Selecting**.
4. Go to the **Data Files** tab. Select **NetCDF**. Save the file locally and rename it *h500_monthly.nc*.
5. Lastly, do the same thing (you'll need to adjust the above directions) for the 1991-2020 *climatology*. Save the file locally and rename it *h500_monthlyclimo.nc*.

Add these to whatever directory you are operating out of with your script. 

In [ ]:
# Import all necessary packages

import numpy as np 
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import xarray as xr
import pandas as pd

In [ ]:
# Using xarray, import the monthly 500 mb heights ranging from 1991-2020. NOTE: YOU WILL NEED TO EITHER NAME THE FILE THE SAME AS THE CODE OR ALTER THE CODE TO FIT YOUR FILE NAME.
filepath='h500_monthly.nc'
ds=xr.open_dataset(filepath, decode_times=False)

#Using xarray, import 1991-2020 climatology of 500 mb heights
filepath='h500_monthlyclimo.nc'
ds_climo=xr.open_dataset(filepath, decode_times=False)

In [ ]:
# viewing our dataset

x = '''
Xarray allows us to view all of the components contained within a NetCDF file. We can also work directly with the data and store multiple variables in one dataset, making them quite convenient. 

We opened our NetCDF as a dataset, rather than a data array. Datasets are typically reserved for files with numerous variables on the same spatial and temporal grid, while data arrays are used for single
variables. However, we will be placing our data in a dataset here.

First, we can view the dimensions of the data. This file contains our height data in 4D: latitude x longitude x pressure x time. So, imagine a 4D-matrix where each of these things are the dimensions in 
which our height data are stored. 

Second, we have our coordinates, which includes more than just spatial coordinates. This aggregates the information outside of the data itself and are the variables that go into the dimensional structure 
of the data. Here, we have Y (latitude), X (longitude), P (pressure level), and T (time). The coordinate name is how we would call the data, e.g., if we typed ds.Y.values, we would get a Numpy array 
containing all latitudes in the dataset. If you typed ds.latitude.values, you would get an error because "latitude" is not recognized as a coordinate within the NetCDF value. Also, notice that P is a 
singleton dimension. These can be annoying. Essentially, they add an extra dimension that we do not need since there is only one element in that dimension. If we were looking at multiple levels, which in
this case we are not, then having a 4D-variable would make more sense. To remove singleton dimensions, use np.squeeze. This will convert the variable to a 3D-var by removing the singleton dimension. 

    All of the data for the Y, X, and P coordinates should be fairly straightforward to all of you. T, on the other hand, may look a bit peculiar (why is time listed as 372.5?). See below for more info. 

Third, we have our actual data contained in the variable "phi". If you want to pull these data, you must use the exact name of the variable (typing "heights" or "500 mb heights") would not work because
that is not how the variable is named in the NetCDF. 
'''

# Uncomment the following command to view the dataset:

# ds

In [ ]:
# viewing "phi"

x='''
Outputting ds.phi will show more information about the data itself. We can see the coordinates that define it (see above) as well as the order of its dimensions (T,P,Y,X), number of values (we have
almost 4 million here), and its attributes. Attributes contain a lot of helpful information, such as what process went into generating the data, names, maximums and minimums, what center produced the 
file, and long name of the variable. 

    The order of the dimensions in phi is very important! This is how you'll call the proper data if using index values. For example, if you wanted to view the first timestep, you would use 
    ds.phi.values[0,:,:,:]. 
'''

# Uncomment the following command to view phi:

# ds.phi

In [ ]:
# viewing "T"

x='''
I mentioned the goofy timesteps above. viewing ds.T will not only show what type of data we have (a data array in this case), but also some of the values and other attributes. Mentioned above, our time 
ranges from 372.5 to 731.5, which is hardly intuitive. If we look at the attributes, however, we can see that the units are in "months since 1960-01-01". Again, that is less than helpful, especially if
we want to pull particular months or years. We can handle this in several different ways, such as succumbing to the date structure contained within our data or defining a new time variable. We will do 
the latter. 
'''

# Uncomment the following command to view T:

# ds.T

In [ ]:
# Map month counts into a calendar

'''
Dates can be incredibly tricky in atmospheric datasets. For example, mapping to/from a Gregorian to/from Julian calendar can shift things slightly. However, we are dealing with monthly data, making it a 
bit easier. We know what months we have, so we can overwrite the current dates contained within T with actual months and years. Once we do this, we can tell Python what month and year we are interested in
and avoid using indices (e.g., figuring out which index corresponds to March of 1997). We can use Pandas to generate DateTime values. 

For climatology, we only need to worry about the month number, since T contains only 12 elements, one for each month. Therefore, we can use simple integers.  
'''

# Generate a new time coordinate and save it to our monthly height dataset
months_since = ds["T"].values #isolate the time values (in months since 1960-1-1)
time_index = pd.date_range("1991-01-01", periods=len(months_since), freq="MS") #create a pandas array starting in 1991-1-1 that maps out monthly dates until 2020-12-1
ds = ds.assign_coords(T=time_index) # put this pandas array into the dataset

# Generate a new time coordinate and save it to our climatology dataset 
ds_climo = ds_climo.assign_coords(T=("T", range(1, 13)))

# Uncomment each of the following individually to check and see if the above code worked:

# ds.T
# ds_climo.T

### **Part 1:** This begins the section where you will need to produce figures to include in your report

Here, we want to produce images for the following:

1. Average 500 mb heights for April, 2003
2. Climatological average 500 mb heights for April
3. 500 mb height anomalies for April, 2003

In [ ]:
# Plotting height data for a single month

'''
Here, we will plot in a way that might look different from what we did in Computer Application in Climate. This is because you can plot xarray data through dataset.plot() 
(e.g., ds.phi[0,0,:,:].plot.contourf()). This is great to quickly view the data but does not allow us to add titles, adjust colorbars, etc. Therefore, we will combine the traditional code used to make
figures with that of xarray. 

UNCOMMENT THE SAVE COMMAND TO SAVE YOUR FIGURE
'''

z500=np.squeeze(ds.phi.sel(T='2003-04')) # save the xarray data into a new variable (which is a DataArray)

vmin, vmax = 5280, 5850  # set upper and lower bounds for the colorbar 

fig=plt.figure(figsize=[10,8])
ax=fig.add_subplot(111,projection=ccrs.PlateCarree())

# Focus on the U.S.
ax.set_extent([-130, -65, 20, 60], crs=ccrs.PlateCarree())  # lon_min, lon_max, lat_min, lat_max

p = z500.plot.contourf(
    ax=ax, levels=20, cmap="viridis", add_colorbar=True,  cbar_kwargs={"shrink": 0.6}, vmin=vmin, vmax=vmax 
) # plot using the syntax for xarray, adding the things we would like to see in our plot

ax.coastlines()
ax.add_feature(cfeature.BORDERS, linewidth=0.5)
ax.add_feature(cfeature.STATES, linewidth=0.5) 
ax.set_title("500 mb Height – April 2004")
gl = ax.gridlines(draw_labels=True, linewidth=0.5, color="gray", alpha=0.5, linestyle="--")
gl.top_labels = False
gl.right_labels = False

# fig.savefig("avg_heights_april2004", dpi=300, bbox_inches="tight")

plt.show()

In [ ]:
# Plotting climatological height data

'''
Now, let's do the same but for our climatology

UNCOMMENT THE SAVE COMMAND TO SAVE YOUR FIGURE
'''

z500_climo=np.squeeze(ds_climo.phi.sel(T=4)) # note that we select 4 because we aren't dealing with DateTimes and we saved 4 to indicate April.   

vmin, vmax = 5280, 5850 

fig=plt.figure(figsize=[10,8])
ax=fig.add_subplot(111,projection=ccrs.PlateCarree())

# Focus on the U.S.
ax.set_extent([-130, -65, 20, 60], crs=ccrs.PlateCarree())  # lon_min, lon_max, lat_min, lat_max

p = z500_climo.plot.contourf(
    ax=ax, levels=20, cmap="viridis", add_colorbar=True,  cbar_kwargs={"shrink": 0.6}, vmin=vmin, vmax=vmax #, "label": "Geopotential Height (m)"}
)

ax.coastlines()
ax.add_feature(cfeature.BORDERS, linewidth=0.5)
ax.add_feature(cfeature.STATES, linewidth=0.5)  # show U.S. states
ax.set_title("Average April 500 mb Height – 1991 to 2020")
gl = ax.gridlines(draw_labels=True, linewidth=0.5, color="gray", alpha=0.5, linestyle="--")
gl.top_labels = False
gl.right_labels = False

# fig.savefig("avg_heights_climo", dpi=300, bbox_inches="tight")

plt.show()

In [ ]:
# plotting anomalies 

x=''' 
Using the code above, compute the anomalies and plot them in a similar way. I have included some code below to help you make your plots look nicer. Be sure you put it in the correct areas, as they are
in a list and will be scattered throughout your cell. 

vmin, vmax = -100, 100 
cb_ticks=np.arange(-100,101,20)
ax=ax, levels=20, cmap="bwr", add_colorbar=True, vmin=vmin, vmax=vmax, cbar_kwargs={"shrink": 0.6, "label": "Geopotential height [gpm]", "ticks": cb_ticks}
'''

# write your code below. 


### **Part 2:**

Do the same thing as Part 1, but for December, 2020. Include all six figures in your write up. 